In [1]:
#TARGIL No. 1 
# By Sahar Goelman
# selected database: rasturants.csv
#TODO: add link to the resource

In [2]:
%pylab inline
import os
import datetime


Populating the interactive namespace from numpy and matplotlib


In [3]:
## Validate that the database file exists otherwise break
if  not os.path.isfile("tmp.csv"):
    print("please add resturants.csv to :"+os.getcwd())

In [1]:
#question (1) loading the database
csv = sc.textFile("tmp.csv")

csv_header = csv.first() 
print("Column names:",format(csv_header))
csv_data = csv.filter(lambda line : line != csv_header) #remove csv headers from data
print("Number of rows loaded: {}".format(csv.count()))

Column names: rand_code


Number of rows loaded: 53577


In [7]:
class FiveHeaders:
    def __init__(self):
        self.values = []
        self.csv_header = dict()
    def add_headers(self, key, val):
        self.csv_header[key] = val
    def get_header(self, key):
        return self.csv_header[key]
    def get_all_values(self):
        self.values = list(self.csv_header.values())
        return self.values
    def get_index_from_key(self, key):
        index_in_header = self.get_header(key)
        self.values.index(index_in_header)
        return self.values.index(index_in_header)


split_headers= csv_header.split(",")
fiveHeaders = FiveHeaders()
headers = ['rand_code'] #, 'business_id','business_id','business_id','business_id']
# headers = ['inspection_score', 'business_id','inspection_id','business_postal_code', 'business_phone_number']
for header in headers: 
     fiveHeaders.add_headers(header, split_headers.index(header))

# fiveHeaders.get_header('inspection_score')
fiveHeaders.get_all_values()



[0]

In [9]:
#problem number (2):

#RDD is assumed to contain an (unique) index column at position 0
def get_RDD_row_by_index(rdd, index=0):
    nrows = rdd.count()
    if index < nrows:
        return rdd.filter(lambda kv: kv[0] == index)
    return None

#RDD is assumed to contain the requested col_nama
def get_RDD_col_values(rdd, col_name):
    col_index = fv.get_index_from_key(col_name)
    return rdd.map(lambda line: line[1][col_index])


#add a unique_id and split csv using commas
baseData = csv_data.zipWithIndex().map(lambda line: (line[-1],line[:-1][0].split(",")))
print("Added unique id at column 0:\nfirst=: {}".format(baseData.first()))
# print("Added unique id at column 0:\nfirst[1]: {}".format(baseData.first()[1]))
# print("Added unique id at column 0:\nfirst[1][1]: {}".format(baseData.first()[1][1]))
#test our method to get the n'th row from RDD
#print("take the {}th row from Data: {}".format(75, get_RDD_row_by_index(baseData,75).first()))
print('number of rows in dataBase: {}'.format(baseData.count()))


Added unique id at column 0:
first=: (0, ['94119'])


number of rows in dataBase: 53576


In [11]:
#problem number (3.a): unique values of the 5 columns 
#RDD is assumed to contain the requested col_name
#row 0 is the first row
def get_RDD_col_value(rdd, row_index, col_name):
    print ("header indexes are:{}".format(fiveHeaders.get_all_values()))
    col_index = fiveHeaders.get_index_from_key(col_name)
    row = get_RDD_row_by_index(rdd, row_index)
    val  = row.first()[1][col_index]
    print ("rdd[{}][{}] = {}".format(row_index, col_index,val))
    #val = row[col_index]
    #print("value of {} at row {} = {}".format(col_name, row_index, val))
    #return val
    #print ("==>Col index of {} is {}".format(col_index, col_name))
    #return rdd.map(lambda line: line[1][col_index])
    #print(get_RDD_col_values(baseData,'business_name'))
    #for col_name in fv.get_index_from_key()
    #dictionary = {}
    #for col_index in fiveHeaders.get_all_values():
    #    baseData.map(lambda row: get_RDD_col_values(baseData, col_index))

get_RDD_col_value(baseData, 100, 'rand_code')

header indexes are:[0]


rdd[100][0] = 94111


In [12]:
import math
#alloows counting only numerical columns
class Data_Filter():
    def __init__(self, header_name):
        self.col_indexes = header_name.get_all_values()
        self.build_line = self._builder(self.list_creator)
        self.build_filter = self._builder(self.create_filter_is_num)
        
    def _builder(self, func):
        return func(self.col_indexes)

    def number(self, s):
        ret = None
        try:
            ret = int(s)
        except ValueError:
            ret = float(s)
        finally:
            return ret

    def is_not_number(self, s):
        ret = False
        try:
            #print("value = ".format(s))
            if (math.isnan(self.number(s))):  #not a number
                ret = True
        except:
            print("exception in is_not_number, s = {}".format(s))
            ret = True
        finally:
            return ret

    def list_creator(self, col_indexes):
        def get_unique_line(_line, unique_id, append_num=True):
            items = []
            for col in col_indexes:
                if append_num:
                    item = self.number(_line[col])
                else:
                    item = _line[col]
                items.append(item)
            return (unique_id, items)
        return get_unique_line


    def create_filter_is_num(self, col_indexes):

        def check_is_number(_line):
            ret = True
            for col in col_indexes:
                try:
                    #print ("_line = {}".format(_line))
                    #print ("value[{}]: {}".format(col, _line[col]))
                    if (math.isnan(self.number(_line[col]))):
                        ret = False
                except:
                    #TODO
                    print('create_filter_is_num caught an exception\n _line[{}]={}'.format(col, _line[col]))
                    ret = False
            return ret
        return check_is_number

print ("filtering data")
filtered_data = Data_Filter(fiveHeaders)
print(filtered_data)
print("end filtering data")

filtering data
end filtering data


In [13]:
i = 0
for col_name in headers: 
    print("index of col_names[{}] = {}".format(col_name,filtered_data.col_indexes[i]))
    i = i + 1
    
filteredByColumn = baseData.filter(lambda line: filtered_data.build_filter(line[1])).map(lambda line: filtered_data.build_line(line[1], line[0]))

def count_distinct(filter_by_column, headerName, key):
    res =  filter_by_column.map(lambda pair: pair[1][headerName.get_index_from_key(key)]).distinct().count()
    return res
def print_distinct(col_name):
        cnt = count_distinct(filteredByColumn, fiveHeaders, col_name)
        print('Distint values of col: {} = {}'.format(col_name, cnt))
        return cnt

index of col_names[rand_code] = 0


In [14]:
#Question 3/a
col_distinct_counts = {}
for header in headers:
    res = print_distinct(header)
    col_distinct_counts[header] = res


Distint values of col: rand_code = 52


In [15]:
#Utility functions to plot histogram
class Histogram():

    def show(self, col_name, color):
        obj = self.countAggregateByKey(col_name)
        self.plot(obj,color)
    
    def countAggregateByKey(self, col_name):
        aggragateByKey = col_name.countByValue() 
        keys  = aggragateByKey.keys()
        x_axis = np.array(sorted(keys))
        y_axis = np.array([aggragateByKey.get(key) for key in x_axis])
        return self.buildObj(x_axis,y_axis)
    
    def buildObj(self, x_axis,y_axis):
        res = dict()
        res['x'] = {
            'pos': np.arange(len(x_axis)),
            'x_axis':x_axis
        }
        res['y']={
            'y_axis':y_axis
        }
        return res
    
    def plot(self, obj,color):
        y_axis = obj['y']['y_axis']
        pos = obj['x']['pos']
        x_axis = obj['x']['x_axis']

        ax = plt.axes()
        ax.set_xticks(pos)
        ax.set_xticklabels(x_axis)

        plt.bar(pos, y_axis, 0.5, color=color)
        plt.xticks(size=16)

        fig = matplotlib.pyplot.gcf()
        fig.set_size_inches(19, 5)
        plt.show()
    

#Question 3/b drawing histograms of the 5 columns
# get all indexes for a specific col name
# we normalize the "inpection_score" variable so that 
def filterColumnByHeader(filteredByColumn,col_names,key):
    return filteredByColumn.map(lambda pair: pair[1][col_names.get_index_from_key(key)])

histogram_colors = ['red', 'green', 'blue', 'yellow', 'magenta']
i = 0
h = Histogram()
for header in headers:
    data = filterColumnByHeader(filteredByColumn,fiveHeaders,header)#
    print ("Histogram of column:{}".format(header))
    h.show(data, histogram_colors[i])
    i = i + 1


Histogram of column:rand_code


NameError: name 'np' is not defined

In [19]:
##############################################################################################
#Utility functions 
##############################################################################################

In [16]:
class DataRecoveryModel:
    def __init__(self,v,sigma,median):
        items = dict()
        items['normal'] = {'mu':v[0]/v[1],'sigma':sigma }
        items['median'] = median
        self.items = items
    def getByType(self,_type):
        return self.items[_type]

In [64]:
class DataRecoveryService():
    def __init__(self, baseData, _fiveHeaders):
        self.fiveHeaders = _fiveHeaders
        self.items = dict()
        #hd = self.hd = Data_Filter(fiveHeaders)
        hd = self.hd = Data_Filter(_fiveHeaders)
        self.shrinkTableAndDarity = baseData.map(lambda line: hd.build_line(line[1], line[0], False))
        self.filteredByColumn = baseData.filter(lambda line: hd.build_filter(line[1])).map(
            lambda line: hd.build_line(line[1], line[0]))

    def fixByType(self, typeFixer, cloumnName):
        dataRecoveryModel = self.items[cloumnName]
        if typeFixer == 'mean':
            metadata = dataRecoveryModel.getByType('normal')
            return metadata['mu']
        elif typeFixer == 'median':
            return dataRecoveryModel.getByType('median')
        else:
            return 0


    def findTheMidle(self, NumOfVal):
        if NumOfVal % 2 != 0:
            NumOfVal = NumOfVal + 1
        return NumOfVal / 2

    def calcDataRecoveryTypes(self, columnName):
        hd = self.hd
        fv = self.fiveHeaders
        v = self.filteredByColumn.map(lambda pair: (pair[1][fv.get_index_from_key(columnName)], 1)).reduce(
            lambda a, b: (a[0] + b[0], a[1] + b[1]))
        n = self.findTheMidle(v[1])
        median = self.filteredByColumn.map(lambda pair: (pair[1][fv.get_index_from_key(columnName)])) \
            .sortBy(lambda item: item).zipWithIndex().filter(lambda pair: pair[1] == n).collect()[0][0]
        self.items[columnName] = DataRecoveryModel(v, 1, median)

    def normelizeColumn(self, cloumnName, sigma):
        fv = self.fiveHeaders
        dataRecoveryModel = self.items[cloumnName]
        metadata = dataRecoveryModel.getByType('normal')
        mu = metadata['mu']
        return self.filteredByColumn.map(
          lambda pair: normalizeFixer(pair[1][fv.get_index_from_key(cloumnName)], mu, sigma))
    
    def fixCellsByType(self, typeFixer, columnName):
        fv = self.fiveHeaders
        hd = self.hd
        fixedValue = self.fixByType(typeFixer, columnName)
        return self.shrinkTableAndDarity.map(
                lambda pair: fixCellsByTypePrivate(pair[1][fv.get_index_from_key(columnName)], fixedValue, hd))

def normalizeFixer(x, mu, sigma):
    return (x - mu)/sigma    
    
def fixCellsByTypePrivate(cell,fixedValue,hd):
    isNotANumber = True
    isNotANumber = hd.is_not_number(cell)
    if isNotANumber :
        cell = fixedValue
    else: 
        cell = hd.num(cell)
    return cell
        

In [22]:
fv = FiveHeaders()
fv.add_headers('rand_code', 0)
print(fv.get_all_values())

#DRService = DataRecoveryService(baseData, fiveHeaders)
DRService = DataRecoveryService(baseData, fv)

hs = Histogram()
i = 0
DRService.calcDataRecoveryTypes(col_name)
print("Histogram of column:{}".format(col_name))
hs.show(DRService.fixCellsByType('mean', col_name), histogram_colors[i])
i = i + 1



[0]


NameError: name 'DataRecoveryService' is not defined